In [ ]:
# if this does not work, separate the 5 lines into 5 cells and run
pip install deepspeech
pip install librosa
pip install sounddevice
pip install prettytable
pip install thinkx
pip install pydub

In [1]:
checkin = "where is the check in desk"
parents = "i have lost my parents"
suitcase = "please i have lost my suitcase"
what_time = "what time is my plane"
where = "where are the restaurants and shops"
your_sentence1 = "this is too much"
your_sentence2 = "i want to die"

en = [checkin, parents, suitcase, what_time, where, your_sentence1, your_sentence2]

checkin_it = "dove e il bancone"
parents_it = "ho perso i miei genitori"
suitcase_it = "per favore, ho perso la mia valigi"
what_time_it = "a che ora e il mio aereo"
where_it = "dove sono i ristoranti ei negozi"

it = [checkin_it, parents_it, suitcase_it, what_time_it, where_it]

checkin_es = "dónde están los mostradores"
parents_es = "he perdido a mis padres"
suitcase_es = "por favor, he perdido mi maleta"
what_time_es = "a que hora es mi avion"
where_es = "donde estan los restaurantes y las tiendas"

es = [checkin_es, parents_es, suitcase_es, what_time_es, where_es]

In [2]:
def wer(transcription, hypothesis):
    tranarr = []
    hypoarr = []
    for word in transcription.split(" "):
        tranarr.append(word)
    for word in hypothesis.split(" "):
        hypoarr.append(word)
        
    counter = 0
    n = len(tranarr)
    if len(tranarr) == len(hypoarr): #if same array length
        outliers = np.setdiff1d(tranarr,hypoarr)
        for i in range(len(tranarr)):
            if tranarr[i] != hypoarr[i]:
                counter += 1
        else:
            return len(outliers)/len(tranarr)*100
    else:
        if len(tranarr) > len(hypoarr): #if hypothesis is shorter than transcription
            correct = 0
            outliers = np.setdiff1d(tranarr,hypoarr)
            for i in range(len(tranarr)):
                for j in range(len(hypoarr)):
                    if tranarr[i] == hypoarr[j]:
                        correct += 1
            return (len(tranarr)-correct)/len(tranarr)*100
            if counter > len(tranarr)+1:
                return len(outliers)/len(tranarr)*100
        elif len(tranarr) < len(hypoarr): #if hypothesis is longer than transcription
            d = len(hypoarr) - len(tranarr)
            counter += d
            outliers = np.setdiff1d(tranarr,hypoarr)
            for i in range(len(tranarr)):
                if tranarr[i] != hypoarr[i]:
                    counter += 1
                else:
                    return len(outliers)/len(tranarr)*100

    return counter/n*100

In [3]:
from deepspeech import Model, version
from prettytable import PrettyTable
import librosa as lr
import numpy as np
import os

def fileRead(dir):
    table = PrettyTable(["Language","File","WER"])
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(filepath):
            audio_file = filepath
            ds = Model(model)
            ds.enableExternalScorer(scorer)
            desired_sample_rate = ds.sampleRate()
            audio = lr.load(audio_file, sr=desired_sample_rate)[0]
            audio = (audio * 32767).astype(np.int16) # scale from -1 to 1 to +/-32767
            res = ds.stt(audio)
            #res = ds.sttWithMetadata(audio, 1).transcripts[0]
            strip1 = filepath.strip("../Audio_Files/")
            if strip1.startswith("EN"):
                lang = "English"
            elif strip1.startswith("ES"):
                lang = "Spanish"
            elif strip1.startswith("IT"):
                lang = "Italian"
            filepath1 = strip1[3:]
            transcription = filepath1[:-4]
            if lang == "English":
                if transcription.startswith("checkin"):
                    tablewer = wer(en[0],res)
                elif transcription.startswith("parents"):
                    tablewer = wer(en[1],res)
                elif transcription.startswith("suitcase"):
                    tablewer = wer(en[2],res)
                elif transcription.startswith("what_time"):
                    tablewer = wer(en[3],res)
                elif transcription.startswith("where"):
                    tablewer = wer(en[4],res)
                elif transcription.startswith("your_sentence1"):
                    tablewer = wer(en[5],res)
                elif transcription.startswith("your_sentence2"):
                    tablewer = wer(en[6],res)
            if lang == "Spanish":
                if transcription.startswith("checkin"):
                    tablewer = wer(es[0],res)
                elif transcription.startswith("parents"):
                    tablewer = wer(es[1],res)
                elif transcription.startswith("suitcase"):
                    tablewer = wer(es[2],res)
                elif transcription.startswith("what_time"):
                    tablewer = wer(es[3],res)
                elif transcription.startswith("where"):
                    tablewer = wer(es[4],res)
            if lang == "Italian":
                if transcription.startswith("checkin"):
                    tablewer = wer(it[0],res)
                elif transcription.startswith("parents"):
                    tablewer = wer(it[1],res)
                elif transcription.startswith("suitcase"):
                    tablewer = wer(it[2],res)
                elif transcription.startswith("what_time"):
                    tablewer = wer(it[3],res)
                elif transcription.startswith("where"):
                    tablewer = wer(it[4],res)
            print(res)
        table.add_row([str(lang), str(filepath1), tablewer])
    print('End of read')
    print(table)

In [4]:
import librosa as lr
import soundfile as sf
from thinkdsp import read_wave, Wave
import numpy as np
import pydub
from pydub import AudioSegment

def EN_filter():
    # replacing all files with the one that has moving average lowpass filter applied
    window = np.ones(3)
    window /= sum(window)
    my_sound = read_wave('../Audio_Files/EN/checkin.wav')
    averages = np.convolve(my_sound.ys, window, mode='same')
    sf.write('../Audio_Files/EN/checkin.wav', averages, 16000)

    window = np.ones(3)
    window /= sum(window)
    my_sound = read_wave('../Audio_Files/EN/parents.wav')
    averages = np.convolve(my_sound.ys, window, mode='same')
    sf.write('../Audio_Files/EN/parents.wav', averages, 16000)

    window = np.ones(1)
    window /= sum(window)
    my_sound = read_wave('../Audio_Files/EN/suitcase.wav')
    averages = np.convolve(my_sound.ys, window, mode='same')
    sf.write('../Audio_Files/EN/suitcase.wav', averages, 16000)

    my_sound = read_wave('../Audio_Files/EN/what_time.wav')
    averages = np.convolve(my_sound.ys, window, mode='same')
    sf.write('../Audio_Files/EN/what_time.wav', averages, 16000)

    window = np.ones(1)
    window /= sum(window)
    my_sound = read_wave('../Audio_Files/EN/where.wav')
    averages = np.convolve(my_sound.ys, window, mode='same')
    sf.write('../Audio_Files/EN/where.wav', averages, 16000)
    
    print("Done")
    
def ES_filter():
    import librosa as lr
    import soundfile as sf
    from thinkdsp import read_wave, Wave
    import numpy as np

    # replacing all files with the one that has lowpass filter applied
    window = np.ones(25)
    window /= sum(window)
    my_sound = read_wave('../Audio_Files/ES/checkin_es.wav')
    averages = np.convolve(my_sound.ys, window, mode='same')
    sf.write('../Audio_Files/ES/checkin_es.wav', averages, 16000)

    window = np.ones(1)
    window /= sum(window)
    my_sound = read_wave('../Audio_Files/ES/parents_es.wav')
    averages = np.convolve(my_sound.ys, window, mode='same')
    sf.write('../Audio_Files/ES/parents_es.wav', averages, 16000)

    window = np.ones(3)
    window /= sum(window)
    my_sound = read_wave('../Audio_Files/ES/suitcase_es.wav')
    averages = np.convolve(my_sound.ys, window, mode='same')
    sf.write('../Audio_Files/ES/suitcase_es.wav', averages, 16000)

    window = np.ones(1)
    window /= sum(window)
    my_sound = read_wave('../Audio_Files/ES/where_es.wav')
    averages = np.convolve(my_sound.ys, window, mode='same')
    sf.write('../Audio_Files/ES/where_es.wav', averages, 16000)


    import pydub
    from pydub import AudioSegment

    sfile = "../Audio_Files/ES/what_time_es.wav"
    aSeg = AudioSegment.from_wav(sfile)
    new = aSeg.low_pass_filter(1600)
    new1 = new.high_pass_filter(200)

    # increase volume
    newsf = new1 + 20
    newsf.export("../Audio_Files/ES/what_time_es.wav", "wav")
    my_sound = read_wave('../Audio_Files/ES/what_time_es.wav')
    my_sound.make_audio()

    window = np.ones(3)
    window /= sum(window)
    my_sound = read_wave('../Audio_Files/ES/what_time_es.wav')
    averages = np.convolve(my_sound.ys, window, mode='same')
    sf.write('../Audio_Files/ES/what_time_es.wav', averages, 16000)

    print("Done")
    
def IT_filter():
    # replacing all files with the one that has lowpass filter applied
    window = np.ones(1)
    window /= sum(window)
    my_sound = read_wave('../Audio_Files/IT/parents_it.wav')
    averages = np.convolve(my_sound.ys, window, mode='same')
    sf.write('../Audio_Files/IT/parents_it.wav', averages, 16000)

    my_sound = read_wave('../Audio_Files/IT/suitcase_it.wav')
    averages = np.convolve(my_sound.ys, window, mode='same')
    sf.write('../Audio_Files/IT/suitcase_it.wav', averages, 16000)

    window = np.ones(2)
    window /= sum(window)
    my_sound = read_wave('../Audio_Files/IT/checkin_it.wav')
    averages = np.convolve(my_sound.ys, window, mode='same')
    sf.write('../Audio_Files/IT/checkin_it.wav', averages, 16000)

    window = np.ones(6)
    window /= sum(window)
    my_sound = read_wave('../Audio_Files/IT/where_it.wav')
    averages = np.convolve(my_sound.ys, window, mode='same')
    sf.write('../Audio_Files/IT/where_it.wav', averages, 16000)

    sfile = "../Audio_Files/IT/what_time_it.wav"
    aSeg = AudioSegment.from_wav(sfile)
    new = aSeg.low_pass_filter(1200)
    new1 = new.high_pass_filter(400)

    # increae volume
    newsf = new1 + 20
    newsf.export("../Audio_Files/IT/what_time_it.wav", "wav")
    my_sound = read_wave('../Audio_Files/IT/what_time_it.wav')
    my_sound.make_audio()
    window = np.ones(1)
    window /= sum(window)
    my_sound = read_wave('../Audio_Files/IT/what_time_it.wav')
    averages = np.convolve(my_sound.ys, window, mode='same')
    sf.write('../Audio_Files/IT/what_time_it.wav', averages, 14000)
    
    print("Done")

/Users/anon/opt/anaconda3/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [8]:
valid = False
while not valid:
    language = input("Select language: 1 English 2 Italian 3 Spanish ")
    try:
        if int(language) == 1: #English
            print("1")
            scorer = "../Models/deepspeech-0.9.3-models.scorer"
            model = "../Models/deepspeech-0.9.3-models.pbmm"
            directory = '../Audio_Files/EN'
            fileRead(directory)
            EN_filter()
            fileRead(directory)
            valid = True
        elif int(language) == 2: #Italian
            print("2")
            scorer = "../Models/kenlm_it.scorer"
            model = "../Models/output_graph_it.pbmm"
            directory = '../Audio_Files/IT'
            fileRead(directory)
            IT_filter()
            fileRead(directory)
            valid = True
        elif int(language) == 3: #Spanish
            print("3")
            scorer = "../Models/kenlm_es.scorer"
            model = "../Models/output_graph_es.pbmm"
            directory = '../Audio_Files/ES'
            fileRead(directory)
            ES_filter()
            fileRead(directory)
            valid = True
        else:        
            print("Error. Please input a number 1 2 or 3")
    except:
        print("Error. Please input a number 1 2 or 3")

Select language: 1 English 2 Italian 3 Spanish 2
2


TensorFlow: v2.3.0-6-g23ad988fcd
DeepSpeech: v0.9.3-0-gf2e9c858


perso i miei genitori


TensorFlow: v2.3.0-6-g23ad988fcd
DeepSpeech: v0.9.3-0-gf2e9c858


nero


...

dove sono ristoranti negozi


TensorFlow: v2.3.0-6-g23ad988fcd
DeepSpeech: v0.9.3-0-gf2e9c858


per favore ho perso la mia valigia


TensorFlow: v2.3.0-6-g23ad988fcd
DeepSpeech: v0.9.3-0-gf2e9c858


dove e il pancone
End of read
+----------+------------------+-------------------+
| Language |       File       |        WER        |
+----------+------------------+-------------------+
| Italian  |  parents_it.wav  |        0.0        |
| Italian  | what_time_it.wav | 71.42857142857143 |
| Italian  |   where_it.wav   | 33.33333333333333 |
| Italian  | suitcase_it.wav  | 28.57142857142857 |
| Italian  | suitcase_it.wav  | 28.57142857142857 |
| Italian  |  checkin_it.wav  |        25.0       |
+----------+------------------+-------------------+
